In [ ]:
import numpy as np
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import sys
import os
sys.path.append(os.path.abspath('..'))

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [84]:

X_data = joblib.load('../data/interim/05_X_new_features_scaled.joblib')
y_data = joblib.load('../data/interim/06_y_combined.joblib')

In [85]:
print(X_data.shape)
print(y_data.shape)

(356224, 91)
(356224, 3)


In [86]:
X_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356224 entries, 0 to 356223
Data columns (total 91 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Time                   356224 non-null  float64
 1   1_red                  356224 non-null  float64
 2   1_blue                 356224 non-null  float64
 3   1_yellow               356224 non-null  float64
 4   2_red                  356224 non-null  float64
 5   2_blue                 356224 non-null  float64
 6   2_yellow               356224 non-null  float64
 7   3_red                  356224 non-null  float64
 8   3_blue                 356224 non-null  float64
 9   3_yellow               356224 non-null  float64
 10  4_red                  356224 non-null  float64
 11  4_blue                 356224 non-null  float64
 12  4_yellow               356224 non-null  float64
 13  5_red                  356224 non-null  float64
 14  5_blue                 356224 non-nu

In [87]:
def remove_highly_correlated_features(X_data, threshold=0.9, verbose=True):
    """
    Returns names of features with reduced correlation (|corr| <= threshold).

    Parameters:
    - X_data (pd.DataFrame): The input DataFrame.
    - threshold (float): Correlation threshold to consider for removal.
    - verbose (bool): If True, prints the pairs and dropped columns.

    Returns:
    - remaining_features (list): List of remaining feature names.
    - dropped_features (list): List of dropped feature names.
    """
    # Compute correlation matrix
    corr_matrix = X_data.corr().abs()

    # Mask the diagonal and lower triangle
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find highly correlated pairs
    high_corr_pairs = [
        (col1, col2, corr_value)
        for col1 in upper.columns
        for col2, corr_value in upper[col1].items()
        if corr_value > threshold
    ]

    # Optionally print the pairs
    if verbose:
        if high_corr_pairs:
            print("Highly correlated feature pairs:")
            for col1, col2, corr_value in high_corr_pairs:
                print(f"{col1} and {col2} : {corr_value:.2f}")
        else:
            print("No feature pairs exceed the correlation threshold.")

    # Identify columns to drop (second column of each pair)
    to_drop = {col2 for col1, col2, corr_value in high_corr_pairs}

    # Get remaining features
    remaining_features = [col for col in X_data.columns if col not in to_drop]

    # Optionally print the dropped and remaining columns
    if verbose:
        print(f"Dropped columns: {list(to_drop)}")
        print(f"Remaining columns: {len(remaining_features)} features")

    return remaining_features, list(to_drop)


# Feature Selection

In [88]:
original_features = X_data.iloc[:, 0:19].columns.tolist()
pca_features = [i for i in X_data.columns if "PCA" in i]
color_features = [i for i in X_data.columns if "color" in i]
rolling_features = [i for i in X_data.columns if "rolling" in i]
temporal_features = [i for i in X_data.columns if "diff" in i]
sensor_features = [i for i in X_data.columns if "block" in i]

print("Original Features:", len(original_features))
print("PCA Features:", len(pca_features))
print("Color Features:", len(color_features))
print("Rolling Features:", len(rolling_features))
print("Temporal Features:", len(temporal_features))
print("Sensor Features:", len(sensor_features))

Original Features: 19
PCA Features: 5
Color Features: 6
Rolling Features: 36
Temporal Features: 18
Sensor Features: 4


In [89]:
feature_set_1 = original_features
feature_set_2 = pca_features
feature_set_3 = original_features + rolling_features + temporal_features 
feature_set_4 = original_features + color_features + sensor_features
feature_set_5 = original_features + pca_features + color_features + rolling_features + temporal_features + sensor_features

### Remove features that are highly corr with one another 

In [98]:
feature_set_6, dropped_columns = remove_highly_correlated_features(X_data, threshold=0.8, verbose=False)

### Feature Importance Ranking + Top-K Selection

In [112]:
def select_top_k_features_with_importance(X_data, y_data, top_k=10, model=None):
    """
    Train a RandomForestRegressor and select top-k important features.
    
    Parameters:
    - X_data (pd.DataFrame): Feature matrix
    - y_data (pd.Series or pd.DataFrame): Target values
    - top_k (int): Number of top features to select
    - model (sklearn model, optional): Provide a pre-configured model if desired
    
    Returns:
    - top_features (list): List of top-k feature names
    - feature_importance_ranking (pd.DataFrame): Full ranking of features and importances
    """
    if model is None:
        model = RandomForestRegressor()
    
    # Fit the model
    model.fit(X_data, y_data)

    # Get importances
    importances = model.feature_importances_
    feature_ranks = sorted(zip(X_data.columns, importances), key=lambda x: x[1], reverse=True)

    # Prepare outputs
    top_features = [f[0] for f in feature_ranks[:top_k]]
    feature_importance_ranking = pd.DataFrame(feature_ranks, columns=['Feature', 'Importance'])

    return top_features, feature_importance_ranking

In [ ]:
feature_set_7, feature_importance_ranking = select_top_k_features_with_importance(X_data, y_data, top_k=10)

print("Top 10 Features:", feature_set_7)
print(feature_importance_ranking.head(10))


In [ ]:
feature_subset_list = [
    feature_set_1, 
    feature_set_2, 
    feature_set_3, 
    feature_set_4, 
    feature_set_5, 
    feature_set_6, 
    feature_set_7
]

for i, subset in enumerate(feature_subset_list):
    print(f"length of feature_set_{i}: ", len(subset))

length of feature_set_0:  19
length of feature_set_1:  5
length of feature_set_2:  73
length of feature_set_3:  29
length of feature_set_4:  88
length of feature_set_5:  33


### Split data

In [109]:
# Split the data into training and validation (80/20 split)
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [111]:
def evaluate_and_save_best_model_generic(
    X_train, y_train, X_val, y_val,
    feature_subsets,
    model_class,
    model_name,
    model_save_dir='../model/',
    multi_output=False,
    **model_kwargs
):
    """
    Generic function to evaluate multiple feature subsets with optional multi-output support,
    save the best model, and report performance.

    Parameters:
    - X_train, y_train, X_val, y_val: Pre-split data
    - feature_subsets (list): List of feature name lists
    - model_class: Scikit-learn model class (e.g., RandomForestRegressor)
    - model_name (str): Name to tag the saved model file
    - model_save_dir (str): Directory to save model
    - multi_output (bool): Whether to wrap with MultiOutputRegressor
    - model_kwargs: Additional model parameters

    Returns:
    - best_model: Trained best model
    - best_metrics (dict): R2, MAE, RMSE
    - best_features (list): Features used
    """
    best_r2 = -np.inf
    best_model = None
    best_metrics = {}
    best_features = []

    for idx, feature_subset in enumerate(feature_subsets, start=1):
        print(f"\nEvaluating Feature Set {idx} with {len(feature_subset)} features using {model_name}...")

        X_train_subset = X_train[feature_subset]
        X_val_subset = X_val[feature_subset]

        # Build model with optional multi-output wrapper
        base_model = model_class(**model_kwargs)
        model = MultiOutputRegressor(base_model) if multi_output else base_model
        
        print(f"Full model parameters:\n{base_model.get_params()}\n")

        # Train and predict
        model.fit(X_train_subset, y_train)
        y_pred = model.predict(X_val_subset)

        # Calculate metrics
        r2 = r2_score(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))

        print(f"R² Score: {r2:.4f} | MAE: {mae:.4f} | RMSE: {rmse:.4f}")

        # Update best model if better
        if r2 > best_r2:
            best_r2 = r2
            best_model = model
            best_metrics = {'R2': r2, 'MAE': mae, 'RMSE': rmse}
            best_features = feature_subset

    # Save best model
    os.makedirs(model_save_dir, exist_ok=True)
    model_path = os.path.join(model_save_dir, f'best_{model_name}.joblib')
    joblib.dump(best_model, model_path)
    print(f"\n✅ Best {model_name} model saved to: {model_path}")

    return best_model, best_metrics, best_features

In [ ]:
# Random Forest
rf_model, rf_metrics, rf_features = evaluate_and_save_best_model_generic(
    X_train, y_train, X_val, y_val,
    feature_subsets=feature_subset_list,
    model_class=RandomForestRegressor,
    model_name='random_forest',
    random_state=42
)

In [ ]:
# Gradient Boosting
gb_model, gb_metrics, gb_features = evaluate_and_save_best_model_generic(
    X_train, y_train, X_val, y_val,
    feature_subsets=feature_subset_list,
    model_class=GradientBoostingRegressor,
    model_name='gradient_boosting',
    random_state=42
)

In [ ]:
# Support Vector Regressor
svr_model, svr_metrics, svr_features = evaluate_and_save_best_model_generic(
    X_train, y_train, X_val, y_val,
    feature_subsets=feature_subset_list,
    model_class=SVR,
    model_name='svr',
    kernel='rbf', C=1.0, epsilon=0.1
)

In [ ]:
# MLP
mlp_model, mlp_metrics, mlp_features = evaluate_and_save_best_model_generic(
    X_train, y_train, X_val, y_val,
    feature_subsets=feature_subset_list,
    model_class=MLPRegressor,
    model_name='mlp',
    random_state=42,
    max_iter=1000,                # Important for convergence
    hidden_layer_sizes=(100, 50), # Example architecture: 2 layers, 100 and 50 neurons
    activation='relu',            # Activation function
    solver='adam',                # Optimizer
    learning_rate='adaptive',     # Learning rate strategy
    early_stopping=True           # Optional: stops if validation score stops improving
)